In [ ]:
import torch
import torchvision

from datasets import load_from_disk
from lerobot.common.datasets.utils import hf_transform_to_torch
# from lerobot.common.datasets.lerobot_dataset import LeRobotDataset, CODEBASE_VERSION
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import datetime
from pathlib import Path
import time


In [ ]:

dataset_path = "../datasets/grasp_100_2024-09-06_17-03-47.hf"
dataset = load_from_disk(dataset_path)
# if "from" not in dataset.column_names:
#   first_frames=dataset.filter(lambda example: example['frame_index'] == 0)
#   from_idxs = torch.tensor(first_frames['index'])
#   to_idxs = torch.tensor(first_frames['index'][1:] + [len(dataset)])
#   episode_data_index={"from": from_idxs, "to": to_idxs}
    
dataset.set_transform(hf_transform_to_torch)
# dataset = dataset.with_format("torch", device=device)


In [ ]:
dataloader = DataLoader(
        dataset,
        num_workers=4,
        batch_size=128,
        shuffle=True,
        # sampler=sampler,
        # pin_memory=params["device"].type != "cpu",
        drop_last=False,
    )


curr_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
dataset_name = Path(dataset_path).stem
CKPT_DIR=f'ckpts/resnet_byol_{dataset_name}_{curr_time}'
TENSORBOARD_DIR=f'runs/resnet_byol_{dataset_name}_{curr_time}'
Path(CKPT_DIR).mkdir(parents=True, exist_ok=True)


In [ ]:
from byol_pytorch import BYOL
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")

# Remove the fc/classification layer
resnet = torchvision.models.resnet18(weights='DEFAULT')
modules = list(resnet.children())[:-1]
backbone = torch.nn.Sequential(*modules)
net = backbone


learner = BYOL(
    net,
    image_size = 240,
    hidden_layer = 8
).to(device)


In [ ]:

opt = torch.optim.Adam(learner.parameters(), lr=3e-4)

writer = SummaryWriter(log_dir=TENSORBOARD_DIR)
epoch = 0
step = 0
for epoch in range(10):
  epoch +=1
  end = time.time()
  for batch in tqdm(dataloader):
      data_load_time = time.time()
      step +=1

      images = torch.cat((batch['observation.pixels.side'], batch['observation.pixels.gripper']), dim=0).to(device)
      gpu_load_time = time.time()

      loss = learner(images)
      pred_time = time.time()

      opt.zero_grad()
      loss.backward()
      opt.step()
      learner.update_moving_average() # update moving average of target encoder
      train_time = time.time()

      writer.add_scalar("Loss/train", loss.item(), step)
      writer.add_scalar("Time/data_load", data_load_time - end, step)
      writer.add_scalar("Time/gpu_transfer", gpu_load_time - data_load_time, step)
      writer.add_scalar("Time/pred_time", pred_time - gpu_load_time, step)
      writer.add_scalar("Time/train_time", train_time - pred_time, step)
      writer.add_scalar("Time/step_time", time.time() - end, step)

# save your improved network
torch.save({'policy_state_dict': net.state_dict(),
            'optimizer_state_dict': opt.state_dict(),
            'loss': loss,
            'epoch': epoch,
            'step': step,
            }, CKPT_DIR + f'/epoch_{epoch}.pt')

In [ ]:
# modules = list(net.children())[:-1]
# backbone = torch.nn.Sequential(*modules).to(device)
dataloader = DataLoader(
        dataset,
        num_workers=4,
        batch_size=128,
        shuffle=False,
        # sampler=sampler,
        # pin_memory=params["device"].type != "cpu",
        drop_last=False,
    )

checkpoint = torch.load("../VINN/ckpts/resnet_byol_grasp_100_2024-09-06_17-03-47_2024-10-12_21-08/epoch_10.pt")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")

# Remove the fc/classification layer
resnet = torchvision.models.resnet18()
modules = list(resnet.children())[:-1]
backbone = torch.nn.Sequential(*modules)
net = backbone
net.load_state_dict(checkpoint["policy_state_dict"])
net = net.to(device)
net.eval()
epoch = checkpoint["epoch"]

# Save embeddings of all images
img_cols = {"observation.pixels.side": "observation.vector.side", "observation.pixels.gripper" : "observation.vector.gripper"}
vec_columns = {v:[] for k,v in img_cols.items()}
for batch in tqdm(dataloader):
    with torch.inference_mode():
      for img_col in img_cols:
        # Batch inference to get embeddings
        embedding = backbone(batch[img_col].to(device)).to("cpu")
        # Add the embedding (size 512) for each image to the column
        vec_columns[img_cols[img_col]].extend([e.numpy() for e in embedding.squeeze()])

# vec_columns = {k: [r.numpy() for r in v] for k, v in vec_columns.items()}
for col in vec_columns:
  assert(len(vec_columns[col]) == len(dataset))
  dataset = dataset.add_column(name=col, column=vec_columns[col])

dataset.set_transform(None)
dataset.save_to_disk(f"../datasets/byol/{dataset_name}_epoch_{epoch}.hf")
dataset.set_format("torch")


In [ ]:
dataset.save_to_disk(f"../datasets/byol/{dataset_name}_epoch_{epoch}.hf")


In [ ]:
# Load dataset and model if not continuing from before
import torch
import torchvision

from datasets import load_from_disk
from tqdm import tqdm
import datetime
from pathlib import Path
import time


dataset_path = "../datasets/byol/grasp_100_2024-09-06_17-03-47_epoch_10.hf"
dataset_name = Path(dataset_path).stem
dataset = load_from_disk(dataset_path)
dataset.set_format("torch")

checkpoint = torch.load("../VINN/ckpts/resnet_byol_grasp_100_2024-09-06_17-03-47_2024-10-12_21-08/epoch_10.pt")
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps")

# Remove the fc/classification layer
resnet = torchvision.models.resnet18()
modules = list(resnet.children())[:-1]
backbone = torch.nn.Sequential(*modules)
net = backbone
net.load_state_dict(checkpoint["policy_state_dict"])
net = net.to(device)


In [ ]:
import gym_lite6.env, gym_lite6.pickup_task, gym_lite6.utils
import gymnasium as gym
import numpy as np
import mediapy as media


task = gym_lite6.pickup_task.GraspAndLiftTask('gripper_left_finger', 'gripper_right_finger', 'box', 'floor')
env = gym.make(
    "UfactoryCubePickup-v0",
    task=task,
    obs_type="pixels_state",
    max_episode_steps=300,
    visualization_width=320,
    visualization_height=240
)
observation, info = env.reset()
media.show_image(env.render())


In [ ]:
import heapq

def dist_metric(x,y):
  """
  L2 norm
  """
  return torch.norm(x-y, dim=1)

def calculate_nearest_neighbours(query, data, k=1, verbose=False):
  t0 = time.time()
  dists = []
  assert len(data[0].shape) == 1, f"Invalid shape, should be 1D: {data.shape}"
  # Takes around 0.25s for 20000 samples
  dists = dist_metric(query, data).cpu()
  heap = [(dists[i].item(), i) for i in range(data.shape[0])]
  
  t1 = time.time()
  heapq.heapify(heap)
  t2 = time.time()

  out = []
  for i in range(k):
    out.append(heapq.heappop(heap))
  
  if verbose:
    print(f"Times: get dists: {t1-t0}, heapify: {t2-t1}")
  return out
        
def calculate_action(dists, dataset, key='action.qpos'):
  """
  Local weighting
  """
  if len(dists) > 1:
    softmin = torch.nn.Softmin(dim=0)
    top_k_weights = softmin(torch.tensor([d[0] for d in dists])).tolist()
    action = sum([top_k_weights[i] * dataset[dists[i][1]][key] for i in range(len(dists))])
  else:
    action = dataset[dists[0][1]][key]

  return action

In [ ]:
observation, info = env.reset()
k=5
frames = []
nn_frames = [[] for _ in range(k)]
action = {}
ep_dict = {"action.qpos": [], "action.gripper": [], "observation.state.qpos": [], "observation.state.qvel": [], "observation.state.gripper": [], "observation.pixels.side": [], "observation.pixels.gripper": [], "reward": [], "timestamp": [], "frame_index": [],}

vector_data = dataset["observation.vector.side"].to(device)

step = 0
done = False
net.eval()
while step < 200:

  t0 = time.time()
  image_side = (torch.from_numpy(observation["pixels"]["side"]).permute(2, 0, 1).unsqueeze(0) / 255).to(device)
  # image_gripper = torch.from_numpy(observation["pixels"]["gripper"]).permute(2, 0, 1).unsqueeze(0) / 255

  with torch.inference_mode():
    embedding_side = net(image_side).squeeze()
    # embedding_gripper = net(image_side, image_gripper).to("cpu")
  t1 = time.time()
  
  nn = calculate_nearest_neighbours(embedding_side, vector_data, k=k)

  t2 = time.time()

  # Save the nearest image in the dataset for comparison
  stacked_frame = np.hstack([observation["pixels"]["side"]] + [dataset[nn[i][1]]["observation.pixels.side"].permute(1, 2, 0).numpy() for i in range(k)])
  frames.append(stacked_frame)

  action["qpos"] = calculate_action(nn, dataset, key='action.qpos').numpy()
  action["gripper"] = round(calculate_action(nn, dataset, key='action.gripper').item())

  
  ep_dict["observation.state.qpos"].append(observation["state"]["qpos"])
  ep_dict["observation.state.qvel"].append(observation["state"]["qvel"])
  ep_dict["observation.state.gripper"].append(observation["state"]["gripper"])
  ep_dict["timestamp"].append(env.unwrapped.data.time)

  # Step through the environment and receive a new observation
  observation, reward, terminated, truncated, info = env.step(action)
  
  # ep_dict["observation.pixels.side"].append(observation["pixels"]["side"])
  # ep_dict["observation.pixels.gripper"].append(observation["pixels"]["gripper"])
  ep_dict["reward"].append(reward)
  ep_dict["action.qpos"].append(action["qpos"])
  ep_dict["action.gripper"].append(action["gripper"])
  # ep_dict["frame_index"].append(step)
  # action = calculate_action(nn, key='observation.state.qvel')
    
  done = truncated | done | terminated
  step += 1

  print(f"Timing: inference: {t1-t0}, nn: {t2-t1}, rest: {time.time()-t2}")


ep_dict["observation.state.qpos"].append(observation["state"]["qpos"])
ep_dict["observation.state.qvel"].append(observation["state"]["qvel"])
ep_dict["observation.state.gripper"].append(observation["state"]["gripper"])
ep_dict["timestamp"].append(env.unwrapped.data.time)

# Append dummy nans to the actions so we have the same number of samples can plot it
ep_dict["action.qpos"].append(np.array([np.nan] * len(action["qpos"])))
ep_dict["action.gripper"].append(np.array([np.nan]))
# ep_dict["reward"].append(np.array([np.nan]))

avg_reward = sum(ep_dict["reward"])/len(ep_dict["reward"])
print(f"Avg reward: {avg_reward}")
media.show_video(frames)



In [ ]:
ep_dict["reward"].append(np.array([np.nan]))


In [ ]:
gym_lite6.utils.plot_dict_of_arrays(ep_dict, "timestamp", keys=["action.qpos", "observation.state.qpos"], sharey=True)

# Qpos search
First search for "similar" qpos in the data, and then of these select most visually similar

In [ ]:
# Testing qpos search
observation, info = env.reset()
print(observation["state"]["qpos"])
qpos_nn = calculate_nearest_neighbours(torch.from_numpy(observation["state"]["qpos"]), dataset["observation.state.qpos"], k=10)
qpos_candidates = [n[1] for n in qpos_nn]
# print(dataset[nn[0][1]]["aobservation.state.qpos"])

# observation, info = env.reset(qpos=action["qpos"])
media.show_image(observation["pixels"]["side"])

frames = []
for n in qpos_nn:
  print(n, dataset[n[1]]["observation.state.qpos"])
  observation, info = env.reset(qpos=dataset[n[1]]["observation.state.qpos"])
  frames.append(observation["pixels"]["side"])
media.show_images(frames)


In [ ]:
# Testing qpos search then vision
image_side = (torch.from_numpy(observation["pixels"]["side"]).permute(2, 0, 1).unsqueeze(0) / 255).to(device)
# image_gripper = torch.from_numpy(observation["pixels"]["gripper"]).permute(2, 0, 1).unsqueeze(0) / 255

with torch.inference_mode():
  embedding_side = net(image_side).squeeze().cpu()

vis_nn = calculate_nearest_neighbours(embedding_side, dataset[qpos_candidates]["observation.vector.side"], k=1)

nn = [(n[0],qpos_candidates[n[1]]) for n in vis_nn]
print(qpos_candidates)
print(nn)

for n in nn:
  print(n[1])
  observation, info = env.reset(qpos=dataset[n[1]]["observation.state.qpos"])
  media.show_image(observation["pixels"]["side"])

print("action: ")
action["qpos"] = calculate_action(nn, dataset, key='action.qpos').numpy()

observation, info = env.reset(qpos=dataset[n[1]]["observation.state.qpos"])
media.show_image(observation["pixels"]["side"])

In [ ]:
for n in nn:
  print(n[1])
  observation, info = env.reset(qpos=dataset[n[1]]["observation.state.qpos"])
  media.show_image(observation["pixels"]["side"])

In [ ]:
observation, info = env.reset()
k=3
frames = []
nn_frames = [[] for _ in range(k)]
action = {}
ep_dict = {"action.qpos": [], "action.gripper": [], "observation.state.qpos": [], "observation.state.qvel": [], "observation.state.gripper": [], "observation.pixels.side": [], "observation.pixels.gripper": [], "reward": [], "timestamp": [], "frame_index": [],}

vector_data = dataset["observation.vector.side"]

step = 0
done = False
net.eval()
while step < 50:

  t0 = time.time()
  image_side = (torch.from_numpy(observation["pixels"]["side"]).permute(2, 0, 1).unsqueeze(0) / 255).to(device)
  # image_gripper = torch.from_numpy(observation["pixels"]["gripper"]).permute(2, 0, 1).unsqueeze(0) / 255

  with torch.inference_mode():
    embedding_side = net(image_side).squeeze().cpu()
    # embedding_gripper = net(image_side, image_gripper).to("cpu")
  t1 = time.time()

  # Find nearest neighbours to qpos
  qpos_nn = calculate_nearest_neighbours(torch.from_numpy(observation["state"]["qpos"]), dataset["observation.state.qpos"], k=30)
  qpos_candidates = [n[1] for n in qpos_nn]
  
  # Find most visually similar
  vis_nn = calculate_nearest_neighbours(embedding_side, dataset[qpos_candidates]["observation.vector.side"], k=k)
  
  nn = [(n[0],qpos_candidates[n[1]]) for n in vis_nn]
  print(nn)
  t2 = time.time()

  # Save the nearest image in the dataset for comparison
  stacked_frame = np.hstack([observation["pixels"]["side"]] + [dataset[nn[i][1]]["observation.pixels.side"].permute(1, 2, 0).numpy() for i in range(k)])
  frames.append(stacked_frame)

  action["qpos"] = calculate_action(nn, dataset, key='action.qpos').numpy()
  action["gripper"] = round(calculate_action(nn, dataset, key='action.gripper').item())
  print(action)

  
  ep_dict["observation.state.qpos"].append(observation["state"]["qpos"])
  ep_dict["observation.state.qvel"].append(observation["state"]["qvel"])
  ep_dict["observation.state.gripper"].append(observation["state"]["gripper"])
  ep_dict["timestamp"].append(env.unwrapped.data.time)

  # Step through the environment and receive a new observation
  observation, reward, terminated, truncated, info = env.step(action)
  
  # ep_dict["observation.pixels.side"].append(observation["pixels"]["side"])
  # ep_dict["observation.pixels.gripper"].append(observation["pixels"]["gripper"])
  ep_dict["reward"].append(reward)
  ep_dict["action.qpos"].append(action["qpos"])
  ep_dict["action.gripper"].append(action["gripper"])
  # ep_dict["frame_index"].append(step)
  # action = calculate_action(nn, key='observation.state.qvel')
    
  done = truncated | done | terminated
  step += 1

  print(f"Timing: inference: {t1-t0}, nn: {t2-t1}, rest: {time.time()-t2}")


ep_dict["observation.state.qpos"].append(observation["state"]["qpos"])
ep_dict["observation.state.qvel"].append(observation["state"]["qvel"])
ep_dict["observation.state.gripper"].append(observation["state"]["gripper"])
ep_dict["timestamp"].append(env.unwrapped.data.time)

# Append dummy nans to the actions so we have the same number of samples can plot it
ep_dict["action.qpos"].append(np.array([np.nan] * len(action["qpos"])))
ep_dict["action.gripper"].append(np.array([np.nan]))
# ep_dict["reward"].append(np.array([np.nan]))

avg_reward = sum(ep_dict["reward"])/len(ep_dict["reward"])
print(f"Avg reward: {avg_reward}")
media.show_video(frames)



In [ ]:
action["qpos"] = calculate_action(nn, dataset, key='action.qpos').numpy()
print(action["qpos"])
for n in nn:
  print(dataset[n[1]]["action.qpos"])

observation, info = env.reset(qpos=action["qpos"])
media.show_image(observation["pixels"]["side"])

# Future actions
Get the action some short time ahead to stop getting stuck in a local minimum

In [ ]:
observation, info = env.reset()
k=3
frames = []
nn_frames = [[] for _ in range(k)]
action = {}
ep_dict = {"action.qpos": [], "action.gripper": [], "observation.state.qpos": [], "observation.state.qvel": [], "observation.state.gripper": [], "observation.pixels.side": [], "observation.pixels.gripper": [], "reward": [], "timestamp": [], "frame_index": [],}

vector_data = dataset["observation.vector.side"]

step = 0
done = False
net.eval()
while step < 50:

  t0 = time.time()
  image_side = (torch.from_numpy(observation["pixels"]["side"]).permute(2, 0, 1).unsqueeze(0) / 255).to(device)
  # image_gripper = torch.from_numpy(observation["pixels"]["gripper"]).permute(2, 0, 1).unsqueeze(0) / 255

  with torch.inference_mode():
    embedding_side = net(image_side).squeeze().cpu()
    # embedding_gripper = net(image_side, image_gripper).to("cpu")
  t1 = time.time()

  # Find nearest neighbours to qpos
  qpos_nn = calculate_nearest_neighbours(torch.from_numpy(observation["state"]["qpos"]), dataset["observation.state.qpos"], k=30)
  qpos_candidates = [n[1] for n in qpos_nn]
  
  # Find most visually similar
  vis_nn = calculate_nearest_neighbours(embedding_side, dataset[qpos_candidates]["observation.vector.side"], k=k)
  
  # HERE is where we look ahead
  # Just going x rows ahead in the data is crude and does not deal with edge case of being at the end of a demo and accidentally skipping ahead to the next one
  # But as a proof of concept it's good enough
  nn = [(n[0],qpos_candidates[n[1]]+10) for n in vis_nn]
  print(nn)
  t2 = time.time()

  # Save the nearest image in the dataset for comparison
  stacked_frame = np.hstack([observation["pixels"]["side"]] + [dataset[nn[i][1]]["observation.pixels.side"].permute(1, 2, 0).numpy() for i in range(k)])
  frames.append(stacked_frame)

  action["qpos"] = calculate_action(nn, dataset, key='action.qpos').numpy()
  action["gripper"] = round(calculate_action(nn, dataset, key='action.gripper').item())
  print(action)

  
  ep_dict["observation.state.qpos"].append(observation["state"]["qpos"])
  ep_dict["observation.state.qvel"].append(observation["state"]["qvel"])
  ep_dict["observation.state.gripper"].append(observation["state"]["gripper"])
  ep_dict["timestamp"].append(env.unwrapped.data.time)

  # Step through the environment and receive a new observation
  observation, reward, terminated, truncated, info = env.step(action)
  
  # ep_dict["observation.pixels.side"].append(observation["pixels"]["side"])
  # ep_dict["observation.pixels.gripper"].append(observation["pixels"]["gripper"])
  ep_dict["reward"].append(reward)
  ep_dict["action.qpos"].append(action["qpos"])
  ep_dict["action.gripper"].append(action["gripper"])
  # ep_dict["frame_index"].append(step)
  # action = calculate_action(nn, key='observation.state.qvel')
    
  done = truncated | done | terminated
  step += 1

  print(f"Timing: inference: {t1-t0}, nn: {t2-t1}, rest: {time.time()-t2}")


ep_dict["observation.state.qpos"].append(observation["state"]["qpos"])
ep_dict["observation.state.qvel"].append(observation["state"]["qvel"])
ep_dict["observation.state.gripper"].append(observation["state"]["gripper"])
ep_dict["timestamp"].append(env.unwrapped.data.time)

# Append dummy nans to the actions so we have the same number of samples can plot it
ep_dict["action.qpos"].append(np.array([np.nan] * len(action["qpos"])))
ep_dict["action.gripper"].append(np.array([np.nan]))
# ep_dict["reward"].append(np.array([np.nan]))

avg_reward = sum(ep_dict["reward"])/len(ep_dict["reward"])
print(f"Avg reward: {avg_reward}")
media.show_video(frames)



In [ ]:
nn